In [3]:
# these variables are defined in the cell above, and 
# influence the rate at which the system progresses
#
# You can tweak these probabilities to create some really cool 
# effects. Try lowering BURNOUT_CHANCE or raising REGROW_ALONE_CHANCE
# to create a self exciting system!

FIRE_SPREAD_CHANCE = .5
BURNOUT_CHANCE = .6
REGROW_ALONE_CHANCE =.03
REGROW_NEIGHBOR_CHANCE = .1
FULLY_REGROW_CHANCE = .05

In [8]:
#Please run this code then click play on the player. Set mode to loop to see it run multiple times. 
#It functions in JupyterLab but not on github. Please contact 
#akazmi30@gatech.edu if you need to see a demo. Thank you! 

import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.animation as animation
import random

N = 100 #Feel free to change this value (50 lets you see the cells more clearly)

# Define states. These influence the color of each state.
# Larger numbers mean brighter colors

ON = 300 # on is burning 
BURNT = 200 # burnt is not actively recovering
GROWING= 100 # is recovering 
OFF = 0 # can be ignited

random.seed(1379)

# populate grid with random on/off - more off than on. Also make sure it is a square grid! 
#cells = np.random.choice([ON,OFF], N*N, p=[0.2, 0.8]).reshape(N, N)

#let's start a fire... (3 lines of fire essentially) 
cells = np.zeros((N, N)).reshape(N, N)
for x in range (1, 30):
    cells.itemset((x, x), ON)
    cells.itemset((2 * x, int(x / 2)), ON)
    cells.itemset((3 * x, int(x / 2)), ON)
    
adj_dict = {}
for x in range(N):
    for y in range(N):
        adj_dict[(x,y)] = [(x - 1, y - 1), (x - 1, y), (x - 1, y + 1),
                          (x, y - 1), (x, y + 1),
                          (x + 1, y - 1), (x + 1, y), (x + 1, y + 1)]

#As before, this function is what takes in one state and gives us the next 
def transition(data):
    global cells
    newGrid = cells.copy()
        
    # this ugly hunk of logic just runs through
    # all the possible states of each hexagon, 
    # and updates its state accordingly
    for x in range(1, N - 1):
        for y in range(1, N - 1):
            curr_state = cells[x][y]
            if curr_state == ON: 
                for neighbor in adj_dict[(x, y)]:
                    if newGrid[neighbor] == OFF and random.random() < FIRE_SPREAD_CHANCE:
                        newGrid[neighbor] = ON
                if random.random() < BURNOUT_CHANCE:
                    newGrid[x][y] = BURNT
            elif curr_state == BURNT and random.random() < REGROW_ALONE_CHANCE:
                newGrid[x][y] = GROWING
            elif curr_state == GROWING:
                for neighbor in adj_dict[(x, y)]:
                    if newGrid[neighbor] == BURNT and random.random() < REGROW_NEIGHBOR_CHANCE:
                        newGrid[neighbor] = GROWING
                if random.random() < FULLY_REGROW_CHANCE:
                    newGrid[x][y] = OFF
        
    cells = newGrid
        
    # update data
    mat.set_data(newGrid)
    cells = newGrid
    return [mat]

# set up animation
fig, ax = plt.subplots(figsize = (6,6))
mat = ax.matshow(cells, cmap="RdYlGn_r")
ani = animation.FuncAnimation(fig, transition, interval=50,
                              save_count=50)
plt.close(fig)
#This code allows us to display the animation
#The animation has been tested to work in JupyterLab and Anaconda's Jupyter Notebooks

#When you first load the notebook you will need to run this cell (ctrl enter) to see the animation player. The image shown is the final state. 

#This will not display from within github.
from IPython.display import HTML
HTML(ani.to_jshtml())